In [9]:
import pandas as pd
import os

In [10]:

directory = r'../data/raw/XRF_data' #location of csv files
counter = 0
for filename in os.listdir(directory): # cycle through csv files
    if filename.endswith(".csv"):
        file_path = os.path.join(directory, filename)        

        date = filename.split("-",1)[1].split(".",1)[0] # get date from filename

        if counter == 0: # create dataframe with 1st file only
            xrf_data = pd.read_csv(file_path, encoding='utf-16', sep="\t")
            xrf_data = xrf_data.iloc[1:] # remove empty row
            xrf_data["date"] = date # add date column

        else: # append for all other files
            df = pd.read_csv(file_path, encoding='utf-16', sep="\t")
            df = df.iloc[1:] # remove empty row
            df["date"] = date # add date column

            xrf_data = xrf_data.append(df) # combine data
        counter += 1

In [11]:
xrf_data = xrf_data.drop(columns=["Analyst", "Field Label 1"]) # drop unnecessary columns
xrf_data = xrf_data.rename(columns={"Field 1": "sample_id"}) # rename sample ID column
xrf_data.loc[xrf_data["sample_id"]=="UNKNOWN", "sample_id"] = "TE2-009" # replace unknown sample ID
xrf_data["date"] = pd.to_datetime(xrf_data["date"], format="%m-%d-%Y").dt.date # make replace dates with date-time object
xrf_data.dropna(subset=["sample_id"], inplace=True) # remove empty rows

In [12]:
xrf_data.head()

,sample_id,Au,Au +/-,As,As +/-,Sb,Sb +/-,Ag,Ag +/-,Al,...,La +/-,Ce,Ce +/-,Pr,Pr +/-,Nd,Nd +/-,Sm,Sm +/-,date
1,oreas45d,4.6,1.2,15.0,2.0,0.0,3.0,49.0,3.0,NaN,...,14.0,20.0,23.0,25.0,24.0,59.0,24.0,-28.0,19.0,2021-10-07
2,oreas504b,6.2,1.2,16.0,2.0,1.0,3.0,42.0,3.0,NaN,...,13.0,-14.0,21.0,-91.0,23.0,82.0,22.0,42.0,17.0,2021-10-07
3,sio2 blank,-0.0,0.6,-1.5,1.0,-0.6,1.9,-28.6,1.7,NaN,...,9.0,-5.0,14.0,30.0,15.0,35.0,15.0,21.0,11.0,2021-10-07
4,OREAS 502b,1.9,1.1,21.0,2.0,-0.0,3.0,43.0,2.0,NaN,...,13.0,81.0,20.0,-74.0,22.0,113.0,21.0,14.0,16.0,2021-10-07
5,TE2-020F,0.4,0.8,3.8,1.9,-1.0,2.0,-56.0,2.0,NaN,...,11.0,17.0,18.0,-33.0,19.0,62.0,18.0,40.0,14.0,2021-10-07


In [13]:
sample_processing_path = "../data/raw/EPSC 552 sample processing.xlsx"

analysis_log = pd.read_excel(sample_processing_path, sheet_name="lab_processing")
sample_list  = pd.read_excel(sample_processing_path, sheet_name="samples")
person_dict  = pd.read_excel(sample_processing_path, sheet_name="person_dictionary")

In [14]:
analysis_log.dropna(subset=["sample_id"],   inplace=True) #remove empty rows
sample_list.dropna( subset=["sample_id"], inplace=True) #remove empty rows
person_dict.dropna( subset=["person_id"],   inplace=True) #remove empty rows

In [15]:
analysis_log["date"] = pd.to_datetime(analysis_log["time_entered"], format="%m-%d-%Y %H:%M:%S.%f").dt.date
analysis_log["analysis_order_index"] = analysis_log["analysis_order_index"].astype("int32") # change dtype from float to integer
analysis_log.drop(analysis_log[analysis_log["process"] != "xrf analysis"].index, inplace=True)

In [16]:
## join to get group ID
analysis_log = pd.merge(analysis_log, person_dict, how="inner", left_on="person_1", right_on="person_id")
analysis_log.drop(columns=["person_1", "person_2", "person_3", "person_id", "description"], inplace=True)


In [17]:
## join to get sample type
analysis_log = pd.merge(analysis_log, sample_list, how="inner", on="sample_id")
analysis_log.drop(columns=["sample_type_2"], inplace=True)
analysis_log.rename(columns={"sample_type_1": "sample_type"}, inplace=True)

In [18]:
analysis_log["comments"] = analysis_log["comments_x"] + "; " + analysis_log["comments_y"]
analysis_log.drop(columns=["process", "time_entered", "last_modified", "comments_x", "comments_y"], inplace=True)

In [19]:
# ensure that sample IDs will match case
xrf_data["sample_id"] = xrf_data["sample_id"].apply(lambda sample_id: sample_id if (sample_id.startswith("GR")) or (sample_id.startswith("TE")) else sample_id.lower())
analysis_log["sample_id"] = analysis_log["sample_id"].apply(lambda sample_id: sample_id if (sample_id.startswith("GR")) or (sample_id.startswith("TE")) else sample_id.lower())

In [20]:
xrf_data = pd.merge(xrf_data, analysis_log, how="inner", on=["sample_id", "date"])

In [21]:
xrf_data.head()

,sample_id,Au,Au +/-,As,As +/-,Sb,Sb +/-,Ag,Ag +/-,Al,...,Nd,Nd +/-,Sm,Sm +/-,date,analysis_order_index,group,sample_type,qaqc_type,comments
0,oreas45d,4.6,1.2,15.0,2.0,0.0,3.0,49.0,3.0,NaN,...,59.0,24.0,-28.0,19.0,2021-10-07,25,group 1,standard,standard,NaN
1,oreas504b,6.2,1.2,16.0,2.0,1.0,3.0,42.0,3.0,NaN,...,82.0,22.0,42.0,17.0,2021-10-07,26,group 1,standard,standard,NaN
2,TE2-020F,0.4,0.8,3.8,1.9,-1.0,2.0,-56.0,2.0,NaN,...,62.0,18.0,40.0,14.0,2021-10-07,29,group 1,soil,field duplicate,NaN
3,GR1-007,3.8,1.1,20.0,4.0,11.0,3.0,-71.0,3.0,NaN,...,84.0,21.0,10.0,16.0,2021-10-07,30,group 1,soil,sample,NaN
4,TE2-011,2.1,0.9,18.0,2.0,1.0,3.0,-51.0,2.0,NaN,...,50.0,19.0,19.0,14.0,2021-10-07,31,group 1,soil,sample,NaN


In [22]:
xrf_data.sort_values(by="analysis_order_index", inplace=True)
xrf_data.reset_index(inplace=True, drop=True)
xrf_data.drop(columns=["analysis_order_index"], inplace=True)

In [23]:
xrf_data.dropna(axis=1, how="all", inplace=True)

In [24]:
xrf_data.head()

,sample_id,Au,Au +/-,As,As +/-,Sb,Sb +/-,Ag,Ag +/-,Ba,...,Pr,Pr +/-,Nd,Nd +/-,Sm,Sm +/-,date,group,sample_type,qaqc_type
0,oreas24b,1.8,0.9,11.1,1.8,-1.0,2.0,32.0,2.0,679.0,...,-71.0,20.0,34.0,20.0,22.0,15.0,2021-10-06,group 2,standard,standard
1,oreas22d,1.0,0.6,0.2,1.0,1.7,2.0,-24.1,1.8,6.0,...,25.0,15.0,-7.0,15.0,24.0,12.0,2021-10-06,group 2,standard,standard
2,oreas24c,2.8,1.0,1.2,1.7,-6.0,3.0,72.0,3.0,256.0,...,-22.0,22.0,37.0,22.0,21.0,18.0,2021-10-06,group 2,standard,standard
3,oreas901,4.8,1.1,70.0,2.0,6.0,2.0,-17.0,2.0,197.0,...,-10.0,19.0,26.0,19.0,7.0,14.0,2021-10-06,group 2,standard,standard
4,oreas501b,2.5,1.0,14.7,1.9,-4.0,2.0,40.0,2.0,951.0,...,-113.0,21.0,104.0,20.0,-26.0,16.0,2021-10-06,group 2,standard,standard


In [25]:
xrf_data.to_excel('../data/interim/xrf_data_clean.xlsx')